FileUpload doesn't work with vscode, please copy resulting database into `/tmp/jnw` directory.

In [1]:
import os
import cub 

from ipywidgets import widgets
from IPython.display import display, clear_output

if not os.path.exists('/tmp/jnw'):
    os.mkdir('/tmp/jnw')
os.chdir('/tmp/jnw')

To analyze results, let's select one algorithm first:

In [2]:
storage = cub.bench.Storage()

# Create a list of options for the dropdown menu
algorithm_list = storage.algnames()

# Create an output widget to display the dataframe
output = widgets.Output()

# Create the dropdown menu widget
dropdown = widgets.Dropdown(
    options=algorithm_list,
    value=algorithm_list[0],
    description='Algorithm:',
    disabled=False,
)

global df

df = storage.alg_to_df(algorithm_list[0])
def on_dropdown_change(change):
    global df
    if change['type'] == 'change' and change['name'] == 'value':
        df = storage.alg_to_df(change['new'])
        with output:
            clear_output()
            # Display the selected dataframe column
            display(df)

# Attach the event handler to the dropdown menu
dropdown.observe(on_dropdown_change)

display(dropdown)

with output:
    display(df)

display(output)

Dropdown(description='Algorithm:', options=('cub.bench.merge_sort.keys', 'cub.bench.merge_sort.pairs', 'cub.be…

Output()

Now we should select a particular CTK, GPU and CUB:

In [4]:
gpus_list = df['gpu'].unique()
ctks_list = df['ctk'].unique()
cubs_list = df['cub'].unique()

global gpu 
global ctk
global cub

gpu = gpus_list[0]
ctk = ctks_list[0]
cub = cubs_list[0]

def create_dropdown_menu(name, options, callback):
    dropdown = widgets.Dropdown(
        options=options,
        value=options[0],
        description=name,
        disabled=False)
    dropdown.observe(callback)
    return dropdown

def on_gpu_change(change):
    global gpu
    if change['type'] == 'change' and change['name'] == 'value':
        gpu = change['new']

def on_ctk_change(change):
    global ctk
    if change['type'] == 'change' and change['name'] == 'value':
        ctk = change['new']

def on_cub_change(change):
    global cub
    if change['type'] == 'change' and change['name'] == 'value':
        cub = change['new']

gpu_dropdown = create_dropdown_menu('GPU:', gpus_list, on_gpu_change)
ctk_dropdown = create_dropdown_menu('CTK:', ctks_list, on_ctk_change)
cub_dropdown = create_dropdown_menu('CUB:', cubs_list, on_cub_change)

dropdown_row = widgets.HBox([gpu_dropdown, ctk_dropdown, cub_dropdown])
display(dropdown_row)

Let's look at the data now

In [5]:
alg_df = df.loc[(df['gpu'] == gpu) & (df['ctk'] == ctk) & (df['cub'] == cub) & (df['center'] != float('inf'))].copy()
alg_df.drop(columns=['gpu', 'ctk', 'cub'], inplace=True)
alg_df

,variant,elapsed,center,samples,T{ct},OffsetT{ct},Elements{io}[pow2],Entropy
0,base,1.644118,0.000040,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",I8,I32,16,1.000
1,ipt_19.tpb_160,1.617769,0.000046,"[3.8912e-05, 3.8912e-05, 3.8912e-05, 3.8912e-0...",I8,I32,16,1.000
2,base,1.672030,0.000039,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",I8,I32,16,0.811
3,ipt_19.tpb_160,1.616526,0.000046,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",I8,I32,16,0.811
4,base,1.664673,0.000040,"[3.6864003e-05, 3.7888003e-05, 3.7888003e-05, ...",I8,I32,16,0.544
...,...,...,...,...,...,...,...,...
3331,ipt_15.tpb_576,1.416653,0.017443,"[0.017391616, 0.017420288, 0.01742848, 0.01743...",I8,I64,28,0.811
3332,ipt_15.tpb_576,1.767652,0.022247,"[0.022086656, 0.022111233, 0.022162432, 0.0221...",I8,I64,28,0.544
3333,ipt_15.tpb_576,2.066945,0.026129,"[0.025906175, 0.0259072, 0.026003456, 0.026004...",I8,I64,28,0.337
3334,ipt_15.tpb_576,2.367239,0.028263,"[0.028086271, 0.02810163, 0.028107777, 0.02821...",I8,I64,28,0.201


In [6]:
bench_columns = ['variant', 'elapsed', 'center', 'samples']
workload_columns = [col for col in alg_df.columns if col not in bench_columns]
# variants = alg_df['variant'].unique()

base_df = alg_df[alg_df['variant'] == 'base'].drop(columns=['variant']).rename(columns={'center': 'base_center', 'samples': 'base_samples'})
base_df.drop(columns=['elapsed'], inplace=True)

merged_df = alg_df.merge(base_df, on=[col for col in alg_df.columns if col in workload_columns])
merged_df['speedup'] = merged_df['base_center'] / merged_df['center']
merged_df = merged_df.drop(columns=['base_center'])
bench_columns.append('speedup')
bench_columns.append('base_samples')
merged_df

,variant,elapsed,center,samples,T{ct},OffsetT{ct},Elements{io}[pow2],Entropy,base_samples,speedup
0,base,1.644118,0.000040,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",I8,I32,16,1.000,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",1.000000
1,ipt_19.tpb_160,1.617769,0.000046,"[3.8912e-05, 3.8912e-05, 3.8912e-05, 3.8912e-0...",I8,I32,16,1.000,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",0.866667
2,ipt_8.tpb_960,1.617064,0.000043,"[3.584e-05, 3.584e-05, 3.584e-05, 3.584e-05, 3...",I8,I32,16,1.000,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",0.928571
3,ipt_14.tpb_320,1.667154,0.000042,"[3.4816e-05, 3.4816e-05, 3.4816e-05, 3.4816e-0...",I8,I32,16,1.000,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",0.951220
4,ipt_10.tpb_576,1.667208,0.000042,"[3.4816e-05, 3.4816e-05, 3.4816e-05, 3.4816e-0...",I8,I32,16,1.000,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",0.951220
...,...,...,...,...,...,...,...,...,...,...
3331,ipt_16.tpb_256,1.616577,0.022057,"[0.022006784, 0.022012929, 0.022017023, 0.0220...",I8,I64,28,0.000,"[0.009255935, 0.009261056, 0.009263105, 0.0092...",0.420706
3332,ipt_20.tpb_448,1.666183,0.022471,"[0.022202367, 0.022203391, 0.022276096, 0.0222...",I8,I64,28,0.000,"[0.009255935, 0.009261056, 0.009263105, 0.0092...",0.412960
3333,ipt_17.tpb_256,1.666830,0.022688,"[0.022679552, 0.022679552, 0.022679552, 0.0226...",I8,I64,28,0.000,"[0.009255935, 0.009261056, 0.009263105, 0.0092...",0.409009
3334,ipt_15.tpb_544,1.666495,0.022356,"[0.022345727, 0.022346752, 0.022349825, 0.0223...",I8,I64,28,0.000,"[0.009255935, 0.009261056, 0.009263105, 0.0092...",0.415079


In [7]:
import functools

ct_axes_values = {}
for col in merged_df.columns:
    if '{ct}' in col:
        ct_axes_values[col] = merged_df[col].unique()

global ct_axes_chosen_values
ct_axes_chosen_values = {}

for ct_axis in ct_axes_values:
    ct_axes_chosen_values[ct_axis] = ct_axes_values[ct_axis][0]

def on_ct_value_change(ct_axis, change):
    global ct_axes_chosen_values
    if change['type'] == 'change' and change['name'] == 'value':
        ct_axes_chosen_values[ct_axis] = change['new']

ct_dropdowns = []
for ct_axis in ct_axes_values:
    ct_dropdowns.append(create_dropdown_menu(ct_axis.replace('{ct}', ''), ct_axes_values[ct_axis], functools.partial(on_ct_value_change, ct_axis)))

ct_dropdown_row = widgets.HBox(ct_dropdowns)
display(ct_dropdown_row)

In [8]:
tuning_df_loc = None 

for ct_axis in ct_axes_values:
    if tuning_df_loc is None:
        tuning_df_loc = (merged_df[ct_axis] == ct_axes_chosen_values[ct_axis])
    else:
        tuning_df_loc = tuning_df_loc & (merged_df[ct_axis] == ct_axes_chosen_values[ct_axis])

tuning_df = merged_df.loc[tuning_df_loc].copy()

for ct_axis in ct_axes_values:
    tuning_df.drop(columns=[ct_axis], inplace=True)

tuning_df

,variant,elapsed,center,samples,Elements{io}[pow2],Entropy,base_samples,speedup
0,base,1.644118,0.000040,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",16,1.000,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",1.000000
1,ipt_19.tpb_160,1.617769,0.000046,"[3.8912e-05, 3.8912e-05, 3.8912e-05, 3.8912e-0...",16,1.000,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",0.866667
2,ipt_8.tpb_960,1.617064,0.000043,"[3.584e-05, 3.584e-05, 3.584e-05, 3.584e-05, 3...",16,1.000,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",0.928571
3,ipt_14.tpb_320,1.667154,0.000042,"[3.4816e-05, 3.4816e-05, 3.4816e-05, 3.4816e-0...",16,1.000,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",0.951220
4,ipt_10.tpb_576,1.667208,0.000042,"[3.4816e-05, 3.4816e-05, 3.4816e-05, 3.4816e-0...",16,1.000,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",0.951220
...,...,...,...,...,...,...,...,...
1819,ipt_24.tpb_1024,1.616816,0.022079,"[0.022068225, 0.022069247, 0.022069247, 0.0220...",28,0.000,"[0.009991168, 0.009993216, 0.009995264, 0.0099...",0.453437
1820,ipt_24.tpb_576,1.716644,0.022847,"[0.02249728, 0.022499328, 0.022675455, 0.02268...",28,0.000,"[0.009991168, 0.009993216, 0.009995264, 0.0099...",0.438195
1821,ipt_22.tpb_128,1.816621,0.024490,"[0.024477696, 0.024480768, 0.024481792, 0.0244...",28,0.000,"[0.009991168, 0.009993216, 0.009995264, 0.0099...",0.408806
1822,ipt_23.tpb_512,1.666552,0.022298,"[0.02228736, 0.022290433, 0.022291455, 0.02229...",28,0.000,"[0.009991168, 0.009993216, 0.009995264, 0.0099...",0.449001


In [9]:
mean_df = tuning_df.groupby(['variant'])['speedup'].mean()
mean_df = mean_df.reset_index()
mean_df.columns = ['variant', 'mean_speedup']
mean_df.sort_values(by=['mean_speedup'], ascending=False, inplace=True)
top_avg_variants = list(mean_df[mean_df['variant'] != 'base'].head(2)['variant'])

result = tuning_df[tuning_df['variant'].isin(top_avg_variants)]
result

,variant,elapsed,center,samples,Elements{io}[pow2],Entropy,base_samples,speedup
43,ipt_14.tpb_352,1.667097,0.000042,"[3.4816e-05, 3.4816e-05, 3.4816e-05, 3.4816e-0...",16,1.000,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",0.951220
46,ipt_17.tpb_256,1.666077,0.000042,"[3.4816e-05, 3.4816e-05, 3.4816e-05, 3.4816e-0...",16,1.000,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",0.951220
119,ipt_14.tpb_352,1.666118,0.000043,"[3.4816e-05, 3.4816e-05, 3.4816e-05, 3.4816e-0...",16,0.811,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",0.904762
122,ipt_17.tpb_256,1.616790,0.000044,"[3.584e-05, 3.6864003e-05, 3.6864003e-05, 3.68...",16,0.811,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",0.883721
195,ipt_14.tpb_352,1.666390,0.000043,"[3.584e-05, 3.584e-05, 3.584e-05, 3.584e-05, 3...",16,0.544,"[3.6864003e-05, 3.7888003e-05, 3.7888003e-05, ...",0.928571
198,ipt_17.tpb_256,1.666235,0.000044,"[3.6864003e-05, 3.6864003e-05, 3.6864003e-05, ...",16,0.544,"[3.6864003e-05, 3.7888003e-05, 3.7888003e-05, ...",0.906977
271,ipt_14.tpb_352,1.666675,0.000043,"[3.584e-05, 3.584e-05, 3.584e-05, 3.584e-05, 3...",16,0.337,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",0.928571
274,ipt_17.tpb_256,1.616154,0.000044,"[3.6864003e-05, 3.6864003e-05, 3.6864003e-05, ...",16,0.337,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",0.906977
347,ipt_14.tpb_352,1.666453,0.000043,"[3.584e-05, 3.584e-05, 3.584e-05, 3.584e-05, 3...",16,0.201,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",0.928571
350,ipt_17.tpb_256,1.616455,0.000045,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",16,0.201,"[3.7888003e-05, 3.7888003e-05, 3.7888003e-05, ...",0.886364


In [20]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

rt_axes_values = {}
for col in merged_df.columns:
    if '{ct}' not in col:
        if col not in bench_columns:
            rt_axes_values[col] = result[col].unique()

rt_axes_values['variant'] = result['variant'].unique()

global rt_axes_chosen_values
rt_axes_chosen_values = {}

for rt_axis in rt_axes_values:
    rt_axes_chosen_values[rt_axis] = rt_axes_values[rt_axis][0]


def displot():
    display_df_loc = None 

    for rt_axis in rt_axes_values:
        if display_df_loc is None:
            display_df_loc = (result[rt_axis] == rt_axes_chosen_values[rt_axis])
        else:
            display_df_loc = display_df_loc & (result[rt_axis] == rt_axes_chosen_values[rt_axis])

    display_df = result.loc[display_df_loc].copy()
    samples = list(display_df['samples'])[0]
    base_samples = list(display_df['base_samples'])[0]

    sns.displot({'base': base_samples, rt_axes_chosen_values['variant']: samples}, aspect=2.)
    plt.show()


def on_rt_value_change(rt_axis, change):
    global rt_axes_chosen_values
    if change['type'] == 'change' and change['name'] == 'value':
        rt_axes_chosen_values[rt_axis] = change['new']
        with plot_output:
            clear_output(wait=True)
            displot()

rt_dropdowns = []
for rt_axis in rt_axes_values:
    rt_dropdowns.append(create_dropdown_menu(rt_axis, rt_axes_values[rt_axis], functools.partial(on_rt_value_change, rt_axis)))

plot_output = widgets.Output()
with plot_output:
    displot()

rt_dropdown_row = widgets.HBox(rt_dropdowns)
display(widgets.VBox([rt_dropdown_row, plot_output]))
